In [3]:
install.packages("leaps")
install.packages("Metrics")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [16]:
library(tidyverse)
library(lubridate)

library(ggplot2)
library(broom)
library(tidyr)
library(dplyr)
library(Metrics)
library(modelr)

library(leaps)
library(caret)

- In the previous assignment, I explored the data with useful visuals that allowed me to observe relationships between input variables and response one and among input variables themselves. I also observed how some input variables were highly correlated (multicollinearity) by plotting a heatmap.
- in this assignment, I will first convert the 'Created_At' and 'Updated_AT' Date variables to numerical values, where they represent the days since created, updated respectively.
- Then, I will clean the Data and use the Forward Selection Algorithm to find the best-fit model.
- Afterward, I am going to split the data into training, testing data with a ratio of 80/20.
- then, I am going to use the variables from the Forward Selection results to fit an Additive linear regression model.
- Then, using the predict function, I am going to predict the number of stars in the testing set using the fitted model.
- Finally, I will calculate the RMSE to evaluate the accuracy of the prediction model.*


In [17]:
df <- read_csv('repositories.csv')

Rows: 215029 Columns: 24
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): Name, Description, URL, Homepage, Language, License, Topics, Defau...
dbl  (5): Size, Stars, Forks, Issues, Watchers
lgl  (9): Has Issues, Has Projects, Has Downloads, Has Wiki, Has Pages, Has ...
dttm (2): Created At, Updated At

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [18]:
df_subset <- df %>% sample_n(size = 10000)

head(df_subset, n = 3)

Name,Description,URL,Created At,Updated At,Homepage,Size,Stars,Forks,Issues,⋯,Has Issues,Has Projects,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch
<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
kvs,💿 KVS: NVMe Key-Value Store,https://github.com/synrc/kvs,2013-05-29 11:10:53,2023-08-31 09:06:02,https://kvs.n2o.dev,917,174,50,0,⋯,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,master
GPU-GEMS-3D-Fluid-Simulation,3D fluid simulation on the in Unity,https://github.com/Scrawk/GPU-GEMS-3D-Fluid-Simulation,2017-06-21 00:49:51,2023-09-14 08:04:33,NA,77,261,39,0,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master
legacy-dotfiles,NA,https://github.com/rbnis/legacy-dotfiles,2018-06-02 11:40:21,2023-09-13 13:30:09,NA,5774,175,44,5,⋯,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,master


In [19]:
df_subset$'Created At' <- as.Date(strptime(df_subset$'Created At', format = "%Y-%m-%d %H:%M:%S"))
df_subset$'Updated At' <- as.Date(strptime(df_subset$'Updated At', format = "%Y-%m-%d %H:%M:%S"))

In [20]:
df_subset_1 <- df_subset %>%
  mutate(repo_age_days = as.numeric(Sys.Date() - `Created At`),
         days_since_update = as.numeric(Sys.Date() - `Updated At`))
head(df_subset_1, n = 3)

Name,Description,URL,Created At,Updated At,Homepage,Size,Stars,Forks,Issues,⋯,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch,repo_age_days,days_since_update
<chr>,<chr>,<chr>,<date>,<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<dbl>,<dbl>
kvs,💿 KVS: NVMe Key-Value Store,https://github.com/synrc/kvs,2013-05-29,2023-08-31,https://kvs.n2o.dev,917,174,50,0,⋯,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,master,4201,455
GPU-GEMS-3D-Fluid-Simulation,3D fluid simulation on the in Unity,https://github.com/Scrawk/GPU-GEMS-3D-Fluid-Simulation,2017-06-21,2023-09-14,NA,77,261,39,0,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master,2717,441
legacy-dotfiles,NA,https://github.com/rbnis/legacy-dotfiles,2018-06-02,2023-09-13,NA,5774,175,44,5,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,master,2371,442


In [21]:
df_subset_2 <- df_subset_1 %>% select(-c('Created At', 'Updated At'))

head(df_subset2, n = 3)

Name,Description,URL,Homepage,Size,Stars,Forks,Issues,Watchers,Language,⋯,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch,repo_age_days,days_since_update
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<dbl>,<dbl>
mm0,Metamath Zero specification language,https://github.com/digama0/mm0,NA,30211,272,40,12,272,Rust,⋯,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,master,2103,437
node-yolo,Node bindings for YOLO/Darknet image recognition library,https://github.com/mobimeo/node-yolo,https://lab.moovel.com/blog/what-you-get-is-what-you-see-nodejs-yolo,1889,370,50,10,370,C++,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master,2869,442
python,Framework files for use with popular python macadmin toolsets,https://github.com/macadmins/python,NA,121,208,45,4,208,Shell,⋯,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,main,1630,433


In [22]:
non_na_repo_age <- df_subset_2 %>% filter(!is.na(repo_age_days))
non_na_days_since_update <- df_subset_2 %>% filter(!is.na(days_since_update))

head(df_subset_2 %>% select(repo_age_days, days_since_update, n = 3))

repo_age_days,days_since_update,n
<dbl>,<dbl>,<chr>
4201,455,https://github.com/synrc/kvs
2717,441,https://github.com/Scrawk/GPU-GEMS-3D-Fluid-Simulation
2371,442,https://github.com/rbnis/legacy-dotfiles
4563,464,https://github.com/mendsley/tinystl
2057,439,https://github.com/victorvoid/dinoql
5719,439,https://github.com/ryanmcgrath/twython


In [23]:
df_cleaned <- df_subset_2 %>% filter(!is.na(Stars) & !is.na(Forks) & !is.na(repo_age_days))  

head(df_cleaned, n = 3)

Name,Description,URL,Homepage,Size,Stars,Forks,Issues,Watchers,Language,⋯,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch,repo_age_days,days_since_update
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<dbl>,<dbl>
kvs,💿 KVS: NVMe Key-Value Store,https://github.com/synrc/kvs,https://kvs.n2o.dev,917,174,50,0,174,Erlang,⋯,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,master,4201,455
GPU-GEMS-3D-Fluid-Simulation,3D fluid simulation on the in Unity,https://github.com/Scrawk/GPU-GEMS-3D-Fluid-Simulation,NA,77,261,39,0,261,C#,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master,2717,441
legacy-dotfiles,NA,https://github.com/rbnis/legacy-dotfiles,NA,5774,175,44,5,175,Shell,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,master,2371,442


In [24]:
summary_stats <- df_cleaned %>%
  summarize(
    count = n(),
    avg_stars = mean(Stars, na.rm = TRUE),
    avg_forks = mean(Forks, na.rm = TRUE),
    avg_issues = mean(Issues, na.rm = TRUE),
    avg_repo_age = mean(repo_age_days, na.rm = TRUE),
    avg_days_since_update = mean(days_since_update, na.rm = TRUE)
  )

print(summary_stats)
head(df_cleaned, n = 3)
nrow(df_cleaned)
sum(is.na(df_cleaned))

# A tibble: 1 × 6
  count avg_stars avg_forks avg_issues avg_repo_age avg_days_since_update
  <int>     <dbl>     <dbl>      <dbl>        <dbl>                 <dbl>
1 10000     1092.      230.       37.8        2764.                  453.


Name,Description,URL,Homepage,Size,Stars,Forks,Issues,Watchers,Language,⋯,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch,repo_age_days,days_since_update
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<dbl>,<dbl>
kvs,💿 KVS: NVMe Key-Value Store,https://github.com/synrc/kvs,https://kvs.n2o.dev,917,174,50,0,174,Erlang,⋯,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,master,4201,455
GPU-GEMS-3D-Fluid-Simulation,3D fluid simulation on the in Unity,https://github.com/Scrawk/GPU-GEMS-3D-Fluid-Simulation,NA,77,261,39,0,261,C#,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master,2717,441
legacy-dotfiles,NA,https://github.com/rbnis/legacy-dotfiles,NA,5774,175,44,5,175,Shell,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,master,2371,442


[1] 10000

[1] 10131

*the limitation I faced with the variables I deleted is that they had many missing values. therefore, I decided to keep the same variables I used in previous assignment.*

In [25]:
set.seed(123) # DO NOT CHANGE!

GitHub_sample <-
    df_cleaned %>%
    mutate(id = row_number())

training_GitHub <- 
    GitHub_sample %>%
    slice_sample(prop = 0.80, replace = FALSE)

testing_GitHub <- 
    GitHub_sample %>% 
    anti_join(training_GitHub, by = "id") %>%
    select(-id)

training_GitHub <- 
    training_GitHub %>% 
    select(-id)

head(training_GitHub)
nrow(training_GitHub)

nrow(testing_GitHub)

Name,Description,URL,Homepage,Size,Stars,Forks,Issues,Watchers,Language,⋯,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch,repo_age_days,days_since_update
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<dbl>,<dbl>
awesome-scholarly-productivity,NA,https://github.com/OpenMindClub/awesome-scholarly-productivity,NA,1806,291,24,0,291,NA,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,main,926,440
re,Regular Expression Construction,https://github.com/jimweirich/re,NA,176,181,17,0,181,Ruby,⋯,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,master,5449,559
pinpoint-docker,Official Dockerized components of the Pinpoint,https://github.com/pinpoint-apm/pinpoint-docker,http://pinpoint-apm.github.io/pinpoint/,250118,428,222,24,428,Shell,⋯,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,master,2400,436
datrie,"Fast, efficiently stored Trie for Python. Uses libdatrie.",https://github.com/pytries/datrie,http://pypi.python.org/pypi/datrie/,2205,515,86,36,515,Python,⋯,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master,4526,460
vbagx,Visual Boy Advance GX,https://github.com/dborth/vbagx,http://wiibrew.org/wiki/Visual_Boy_Advance_GX,15445,250,34,2,250,C++,⋯,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master,3450,439
vcrtl,C++ Exceptions in Windows Drivers,https://github.com/avakar/vcrtl,NA,50,183,39,4,183,C++,⋯,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,master,1829,431


[1] 8000

[1] 2000

- One of the limitations I faced when runing the Forward Selection Algorithm on the given data is that The kernel will collapse and the restart entirly.
- To solve this problem, I will drop some of the catogrical variables that have many levels (>2) and I, from the experince I gained in this course belive they are in little to no utillity in predicting the number of stars.

In [27]:
features <- c(repo_age_days, days_since_update, "Size", "Stars", "Forks",
              "Issues", 'Watchers', 'Has Discussions')
df_subset <- df_subset %>% select(all_of(features))

head(df_subset, n = 3)

ERROR: Error in eval(expr, envir, enclos): object 'repo_age_days' not found


In [ ]:
GitHub_forward_sel <- regsubsets(
  Stars ~ ., data = training_GitHub,
  method = "forward",
  nvmax = ncol(training_GitHub) - 1
)
GitHub_forward_sel

GitHub_fwd_summary <- summary(GitHub_forward_sel)

GitHub_fwd_summary <- tibble(
  n_input_variables = 1:length(GitHub_fwd_summary$rss),  
  RSS = GitHub_fwd_summary$rss,
  BIC = GitHub_fwd_summary$bic,
  Cp = GitHub_fwd_summary$cp
)

Warning message in model.matrix.default(terms(formula, data = data), mm):
“term names will be truncated”


In [ ]:
GitHub_fwd_summary
summary(GitHub_forward_sel)

*based on the forward selection, the variables for the best-fit model are Size Forks Issues Watchers. Therefore I will fit the training data into a linear regression prediction model using an additive model for all the variables we just selected(4). the reason I selected model (4) is that it has the best combination of lowest CP, BIC and the among the highest RSS.*

In [53]:
GitHub_full <- lm(Stars ~ Size + Watchers + `Has Discussions`, data = training_GitHub)

GitHub_full


Call:
lm(formula = Stars ~ Size + Watchers + `Has Discussions`, data = training_GitHub)

Coefficients:
          (Intercept)                   Size               Watchers  
            1.596e-13              5.887e-20              1.000e+00  
`Has Discussions`TRUE  
            1.948e-15  


In [54]:
GitHub_test_pred_full <- predict(GitHub_full, newdata = testing_GitHub)

head(GitHub_test_pred_full)

1    2    3    4    5    6 
1848  815  194  257  401  841

In [55]:
GitHub_RMSE_model <- tibble(
    Model = "Full Regression",
    RMSE = rmse(model = GitHub_full,
                data = testing_GitHub))

GitHub_RMSE_model 

Model,RMSE
<chr>,<dbl>
Full Regression,5.781083e-13


*By using the vif function, and to resolve multicollinearity, I got the following results: Size: 1.001, Forks: 1.615, Issues: 1.441, Watchers: 2.042, `Has Discussions`: 1.051repo_age_days: 1.087, days_since_update: 1.071.
and I saw that 'Watchers' has a strong correlation with both 'Forks' and 'Issues' (0.62 and 0.55 respectively). Now lets fit a linear regression model without 'Watchers'*

In [56]:
GitHub_No_Watchers <- lm(Stars ~ Size + Forks + Issues + `Has Discussions`, data = training_GitHub)

GitHub_No_Watchers


Call:
lm(formula = Stars ~ Size + Forks + Issues + `Has Discussions`, 
    data = training_GitHub)

Coefficients:
          (Intercept)                   Size                  Forks  
            4.104e+02              4.346e-05              1.878e+00  
               Issues  `Has Discussions`TRUE  
            2.863e+00              1.029e+03  


In [57]:
GitHub_pred_No_Watchers <- predict(GitHub_No_Watchers, newdata = testing_GitHub)

head(GitHub_pred_No_Watchers)

1         2         3         4         5         6 
1250.3492  770.2662  497.9961  478.1555  598.7147 1480.0298

In [58]:
GitHub_RMSE_No_Watchers <- tibble(
    Model = "Full Regression No Watchers",
    RMSE = rmse(model = GitHub_No_Watchers,
                data = testing_GitHub))

GitHub_RMSE_No_Watchers

Model,RMSE
<chr>,<dbl>
Full Regression No Watchers,1962.103


*Since we saw in the previous assignment visualization that 'Watchers' was perfectly linearly correlated with Stars, let's fit a simple linear regression model with only 'Watchers' as an input.*

In [59]:
GitHub_Simple_model <- lm(Stars ~ Issues, data = training_GitHub)

GitHub_Simple_model


Call:
lm(formula = Stars ~ Issues, data = training_GitHub)

Coefficients:
(Intercept)       Issues  
    845.588        6.472  


In [60]:
GitHub_test_pred_Simple <- predict(GitHub_full, newdata = testing_GitHub)

head(GitHub_test_pred_full)

1    2    3    4    5    6 
1848  815  194  257  401  841

In [61]:
GitHub_RMSE_Simple_models <- tibble(
    Model = "OLS Simple Regression",
    RMSE = rmse(model = GitHub_Simple_model,
                data = testing_GitHub))

GitHub_RMSE_Simple_models

Model,RMSE
<chr>,<dbl>
OLS Simple Regression,2435.912
